In [3]:
# from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.formatting import Rule
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def set_color_modes():
    d_colors = {}
    d_colors["Green-Yellow-Red"] = ("FFF8696B", "FFFFEB84", "FF63BE7B")
    d_colors["Red-Yellow-Green"] = ("FF63BE7B", "FFFFEB84", "FFF8696B")
    d_colors["Red-White-Green"] = ("FF63BE7B", "FFFCFCFF", "FFF8696B")
    d_colors["Green-White-Red"] = ("FFF8696B", "FFFCFCFF", "FF63BE7B")
    d_colors["Blue-White-Red"] = ("FFF8696B", "FFFCFCFF", "FF5A8AC6")
    d_colors["Red-White-Blue"] = ("FF5A8AC6", "FFFCFCFF", "FFF8696B")
    d_colors["Green-White"] = ("FFFCFCFF", "FF63BE7B")
    d_colors["White-Green"] = ("FF63BE7B", "FFFCFCFF")
    d_colors["Red-White"] = ("FFFCFCFF", "FFF8696B")
    d_colors["White-Red"] = ("FFF8696B", "FFFCFCFF")
    d_icons = {}
    d_icons["3 Arrow (Coloured)"] = "3Arrows"
    d_icons["3 Traffic Lights (Unrimmed)"] = "NoneType??"
    d_icons["3 Symbols (Circled)"] = "3Symbols"
    d_icons["3 Flags"] = "3Flags"
    return d_colors, d_icons

def sum_of_digits(i):
    sum_of_digits = 0
    i = str(i)
    for digit in i:
        sum_of_digits += int(digit)
    return sum_of_digits
  
def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    seed = "abc"
    val = hash(str(matr) + seed) % 10000000000
    if ws["BB3"].value == val:
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "          "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"

def check_exercise_1_1(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        # If Auto-Filter are used check the status of the filter
        if ws.auto_filter.ref != None:
            for i in range(len(ws.auto_filter.filterColumn)):
                for j in range(len(ws.auto_filter.filterColumn[i].filters.dateGroupItem)):
                    if int(ws.auto_filter.filterColumn[i].filters.dateGroupItem[j].year) in [2020, 2021, 2022, 2023]:
                        points += 2
                if len(ws.auto_filter.filterColumn[i].filters.filter) == 1:
                    l_bundesl = [str(i) for i in range(1, 17)]
                    l_bundesl += ["0" + str(i) for i in range(1, 10)]
                    if ws.auto_filter.filterColumn[i].filters.filter[0] in l_bundesl:
                        points += 20
                    if ws.auto_filter.filterColumn[i].filters.filter[0] == "00+":
                        points += 200
        # If no Auto-Filters are used, check the content of A2:C2
        else:
            if ws["A2"].value != None:
                if int(str(ws["A2"].value)[:4]) in [2020, 2021, 2022, 2023]:
                    points += 2
            if  ws["B2"].value != None:
                if int(ws["B2"].value) in list(range(1, 17)):
                    points += 20
            if  ws["C2"].value != None:
                if ws["C2"].value == "00+":
                    points += 200
        # Check if there are numbers (int, float) in H2:H11
        colH_is_num = True
        for row_ind in range(2, 12):
            if ws["H" + str(row_ind)].value != None:
                if "str" in str(type(ws["H" + str(row_ind)].value)):
                    colH_is_num = False
            else:
                colH_is_num = False
        if colH_is_num:
            points += 4000

        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_1")
        return "NOK"

def check_exercise_1_2(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        line_chart = ""
        if len(ws._charts) >= 1 and "line_chart" in str(type(ws._charts[0])):
            line_chart = ws._charts[0]
        # Check, if xaxis comes from Col A
        if line_chart != "":
            xaxis = ""
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.strRef != None:  # x-Axis-Ticks should come from Col H
                xaxis = line_chart.series[0].identifiers.strRef.f
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.numRef != None:  # x-Axis-Ticks should come from Col H
                xaxis = line_chart.series[0].identifiers.numRef.f
            if xaxis != "":
                xaxis = xaxis.split("!")[1].replace("$", "")
                cell1, cell2 = xaxis.split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 2
            # Values should come from column H
            values = ""
            if line_chart.series[0].val.numRef != None:
                    values = line_chart.series[0].val.numRef.f
                    values = values.split("!")[1].replace("$", "")
                    if ":" in values:
                        cell1, cell2 = values.split(":")
                        if cell1[0] == "H" and cell2[0] == "H":
                            points += 20

        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"

def check_exercise_2_1(wb_val, matr):
    try:
        ws = wb_val.worksheets[1]
        points = 0
        # Check, if G1 has background color MINT and alignment = center
        if ws["G1"].fill.fgColor.index == "FF00B1AC":
            points += 1
        if ws["G1"].alignment.horizontal == "center":
            points += 10
        # Check if there are numbers (int, float) in Q2:Q11
        colH_is_num = True
        for row_ind in range(2, 12):
            if ws["Q" + str(row_ind)].value != None:
                if "str" in str(type(ws["Q" + str(row_ind)].value)):
                    colH_is_num = False
            else:
                colH_is_num = False
        if colH_is_num:
            points += 100
    
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"

def check_exercise_2_2(wb_val, matr, d_colors, d_icons):
    try:
        ws = wb_val.worksheets[1]
        points = 0
        # Set the conditions according to the content of U1:U3
        color_scheme = ""
        if ws["U1"].value != None:
            color_scheme = ws["U1"].value
        icon = ""
        if ws["U2"].value != None:
            icon = ws["U2"].value
        category = ""
        if ws["U3"].value != None:
            category = ws["U3"].value
        if ws.conditional_formatting != None:
            # Check if the color category is set correctly in B2:P366 (P367)
            print(matr)
            for range, rule in ws.conditional_formatting._cf_rules.items():
                if "B2:P366" in str(range) or "B2:P367" in str(range):
                    if rule[0].colorScale != None:
                        l_color = []
                        for color in rule[0].colorScale.color:
                            l_color.append(str(color.rgb))
                        l_color = sorted(l_color)
                        l1_color = sorted(list(d_colors[color_scheme]))
                        if l_color == l1_color:
                            points += 3
            # Check if the symbols are set correctly in Q2:Q366 (Q367)
            for range, rule in ws.conditional_formatting._cf_rules.items():
                if "Q2:Q366" in str(range) or "Q2:Q367" in str(range):
                    if rule[0].iconSet != None:
                        if rule[0].iconSet.iconSet != None:
                            icon1 = str(rule[0].iconSet.iconSet)
                        else:
                            icon1 = "NoneType??"
                        if icon1 == d_icons[icon]:
                            points += 30
            # Check if "bold-rule" is set in R2:R366 (R367)
            for range, rule in ws.conditional_formatting._cf_rules.items():
                if "R2:R366" in str(range) or "R2:R367" in str(range):
                    if rule[0].dxf != None:
                        if rule[0].dxf.font.bold == True:
                            points += 200
                    if rule[0].formula != None:
                        if str(rule[0].formula[0]) == "$U$3":
                            points += 2000

        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_2")
        return "NOK"

def check_workbook(matr, wb, wb_val):
    points = [matr]
    d_colors, d_icons = set_color_modes()
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb, matr))
    points.append(check_exercise_2_1(wb_val, matr))
    points.append(check_exercise_2_2(wb_val, matr, d_colors, d_icons))
    
    return points

l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
    header = ["MatrNr", "Fingerprint", "Array", "1 Import", "1 LineChart", "2 Import", "2 CondFormat"] 
common.result_into_file(l_points, header)


3726750
3738393
3722088
3712522
FAIL: 3712522 check_exercise_2_2
[3726750, 'NOK', '          ', 10, 4, 3, 5]
[3738393, 'NOK', '          ', 8, 4, 3, 8]
[3722088, 'NOK', '          ', 8, 4, 1, 0]
[3712522, 'NOK', '          ', 10, 4, 1, 'NOK']
['~$3712522.xlsx']


In [ ]:
os.chdir(".")
print(os.getcwd())